# Data Exploration and Preprocessing

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from geopy.distance import geodesic
from scipy import stats

### Basic Data Exploration

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [4]:
data.shape

(1458644, 11)

In [5]:
data.isna().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [6]:
data.duplicated().sum()

0

In [7]:
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

### Data Preprocessing

In [8]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [9]:
data.dtypes

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
dtype: object

In [10]:
data['week_day'] = data.pickup_datetime.dt.strftime('%A')
data['week_day_num'] = data.pickup_datetime.dt.weekday
data['month'] = data.pickup_datetime.dt.month
data['pickup_hour'] = data.pickup_datetime.dt.hour

In [11]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,week_day,week_day_num,month,pickup_hour
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Monday,0,3,17
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Sunday,6,6,0
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Tuesday,1,1,11
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,Wednesday,2,4,19
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,Saturday,5,3,13


In [12]:
distance = []
for index in data['pickup_latitude'].index:
    distance.append(geodesic((data['pickup_latitude'].iloc[index], data['pickup_longitude'].iloc[index]), (data['dropoff_latitude'].iloc[index], data['dropoff_longitude'].iloc[index])).kilometers)

data['distance'] = distance

In [13]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,week_day,week_day_num,month,pickup_hour,distance
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Monday,0,3,17,1.502172
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Sunday,6,6,0,1.808660
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Tuesday,1,1,11,6.379687
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,Wednesday,2,4,19,1.483632
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,Saturday,5,3,13,1.187038


In [14]:
data['speed_m_s'] = (data['distance'] * 1000)/ data['trip_duration']

In [15]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,week_day,week_day_num,month,pickup_hour,distance,speed_m_s
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Monday,0,3,17,1.502172,3.301477
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Sunday,6,6,0,1.808660,2.727994
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Tuesday,1,1,11,6.379687,3.003619
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,Wednesday,2,4,19,1.483632,3.458351
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,Saturday,5,3,13,1.187038,2.728822


In [16]:
data['speed_km_hr'] = (data['distance'] * 3600) / data['trip_duration']

In [17]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,week_day,week_day_num,month,pickup_hour,distance,speed_m_s,speed_km_hr
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,Monday,0,3,17,1.502172,3.301477,11.885316
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,Sunday,6,6,0,1.808660,2.727994,9.820778
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,Tuesday,1,1,11,6.379687,3.003619,10.813029
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,Wednesday,2,4,19,1.483632,3.458351,12.450063
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,Saturday,5,3,13,1.187038,2.728822,9.823760


### Identifying the Outliers

In [18]:
data.dtypes

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
week_day                      object
week_day_num                   int64
month                          int64
pickup_hour                    int64
distance                     float64
speed_m_s                    float64
speed_km_hr                  float64
dtype: object

In [22]:
numerical_columns = ['trip_duration', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'distance', 'speed_m_s', 'speed_km_hr']
for column in numerical_columns:
    z_scores = stats.zscore(data[column])
    threshold = 3  
    outliers = data[abs(z_scores) > threshold]
    print(f"Outliers in column '{column}':")
    print(outliers)

Outliers in column 'trip_duration':
                id  vendor_id     pickup_datetime    dropoff_datetime  \
531      id3307903          2 2016-02-20 04:03:06 2016-02-21 03:33:00   
1134     id1091477          2 2016-05-07 18:36:22 2016-05-08 18:32:11   
1865     id3431345          2 2016-06-07 12:58:48 2016-06-08 12:58:00   
3442     id1487069          2 2016-02-13 00:21:49 2016-02-14 00:19:05   
4172     id3674870          2 2016-03-18 11:54:20 2016-03-19 11:34:17   
...            ...        ...                 ...                 ...   
1455097  id2798196          2 2016-03-20 03:51:36 2016-03-21 00:00:00   
1456458  id2055880          2 2016-02-08 19:11:31 2016-02-09 19:08:44   
1457207  id1910950          2 2016-06-08 16:29:12 2016-06-09 16:11:03   
1457752  id1215198          2 2016-02-02 11:31:10 2016-02-03 11:29:44   
1458076  id2332290          2 2016-04-24 00:58:34 2016-04-25 00:56:16   

         passenger_count  pickup_longitude  pickup_latitude  \
531                    3

Outliers in column 'distance':
                id  vendor_id     pickup_datetime    dropoff_datetime  \
24       id3321406          2 2016-06-03 08:15:05 2016-06-03 08:56:30   
50       id2067879          2 2016-01-19 23:19:01 2016-01-19 23:44:58   
86       id0283075          1 2016-04-06 23:42:26 2016-04-07 00:12:08   
102      id0105610          1 2016-02-19 20:01:24 2016-02-19 20:35:49   
104      id3508035          2 2016-06-06 20:13:56 2016-06-06 20:45:20   
...            ...        ...                 ...                 ...   
1458485  id1896027          1 2016-04-03 05:13:28 2016-04-03 05:40:08   
1458550  id2976426          1 2016-06-06 10:48:13 2016-06-06 11:40:31   
1458571  id1440046          1 2016-06-26 22:59:48 2016-06-26 23:40:42   
1458584  id1607338          2 2016-06-14 20:37:11 2016-06-14 21:16:06   
1458600  id0995846          2 2016-05-09 17:26:56 2016-05-09 18:30:37   

         passenger_count  pickup_longitude  pickup_latitude  \
24                     1     

In [23]:
for column in numerical_columns:
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    print(f"Outliers in column '{column}':")
    print(outliers)

Outliers in column 'trip_duration':
                id  vendor_id     pickup_datetime    dropoff_datetime  \
2        id3858529          2 2016-01-19 11:35:24 2016-01-19 12:10:48   
16       id3379579          2 2016-04-11 17:29:50 2016-04-11 18:08:26   
24       id3321406          2 2016-06-03 08:15:05 2016-06-03 08:56:30   
29       id1410897          1 2016-03-23 14:10:39 2016-03-23 14:49:30   
33       id0023813          1 2016-04-17 14:38:30 2016-04-17 15:16:19   
...            ...        ...                 ...                 ...   
1458571  id1440046          1 2016-06-26 22:59:48 2016-06-26 23:40:42   
1458584  id1607338          2 2016-06-14 20:37:11 2016-06-14 21:16:06   
1458589  id2071901          1 2016-06-10 16:48:20 2016-06-10 17:23:13   
1458600  id0995846          2 2016-05-09 17:26:56 2016-05-09 18:30:37   
1458621  id3267199          2 2016-05-09 14:33:30 2016-05-09 15:12:45   

         passenger_count  pickup_longitude  pickup_latitude  \
2                      1

Outliers in column 'dropoff_latitude':
                id  vendor_id     pickup_datetime    dropoff_datetime  \
24       id3321406          2 2016-06-03 08:15:05 2016-06-03 08:56:30   
27       id2104175          1 2016-06-20 23:07:16 2016-06-20 23:18:50   
31       id2822549          2 2016-05-21 13:03:24 2016-05-21 13:20:52   
67       id3913101          1 2016-06-25 22:05:46 2016-06-25 22:22:29   
115      id2358953          2 2016-06-27 00:31:28 2016-06-27 01:06:44   
...            ...        ...                 ...                 ...   
1458424  id0998702          2 2016-03-06 02:15:18 2016-03-06 02:24:16   
1458457  id0960839          2 2016-04-17 11:22:43 2016-04-17 11:37:41   
1458536  id3423195          2 2016-04-09 04:29:43 2016-04-09 04:57:47   
1458553  id2796154          1 2016-04-21 03:30:00 2016-04-21 03:51:30   
1458629  id3109086          2 2016-06-24 10:33:51 2016-06-24 10:43:52   

         passenger_count  pickup_longitude  pickup_latitude  \
24                   

Outliers in column 'speed_km_hr':
                id  vendor_id     pickup_datetime    dropoff_datetime  \
50       id2067879          2 2016-01-19 23:19:01 2016-01-19 23:44:58   
54       id0526266          2 2016-05-15 09:42:58 2016-05-15 09:56:37   
67       id3913101          1 2016-06-25 22:05:46 2016-06-25 22:22:29   
86       id0283075          1 2016-04-06 23:42:26 2016-04-07 00:12:08   
102      id0105610          1 2016-02-19 20:01:24 2016-02-19 20:35:49   
...            ...        ...                 ...                 ...   
1458553  id2796154          1 2016-04-21 03:30:00 2016-04-21 03:51:30   
1458570  id1608086          1 2016-06-06 11:57:49 2016-06-06 12:04:43   
1458584  id1607338          2 2016-06-14 20:37:11 2016-06-14 21:16:06   
1458640  id1049543          1 2016-01-10 07:35:15 2016-01-10 07:46:10   
1458641  id2304944          2 2016-04-22 06:57:41 2016-04-22 07:10:25   

         passenger_count  pickup_longitude  pickup_latitude  \
50                     1  

In [24]:
z_scores = stats.zscore(data['speed_km_hr'])
threshold = 3 
outliers_zscore = data[abs(z_scores) > threshold]
Q1 = data['speed_km_hr'].quantile(0.25)
Q3 = data['speed_km_hr'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_iqr = data[(data['speed_km_hr'] < lower_bound) | (data['speed_km_hr'] > upper_bound)]
print("Outliers based on Z-Score:")
print(outliers_zscore)
print("\nOutliers based on IQR:")
print(outliers_iqr)

Outliers based on Z-Score:
                id  vendor_id     pickup_datetime    dropoff_datetime  \
991      id1859156          1 2016-02-17 00:54:13 2016-02-17 01:36:27   
1107     id0729156          2 2016-06-23 13:36:48 2016-06-23 13:36:50   
1383     id1904103          2 2016-06-05 04:58:04 2016-06-05 05:23:19   
1492     id2456838          1 2016-01-13 20:11:46 2016-01-13 20:11:53   
1870     id2641288          2 2016-04-03 08:34:32 2016-04-03 08:49:58   
...            ...        ...                 ...                 ...   
1450392  id1169422          2 2016-05-09 07:17:16 2016-05-09 07:29:33   
1452172  id0788889          2 2016-06-26 08:13:52 2016-06-26 08:29:44   
1453346  id1081148          1 2016-05-17 04:14:11 2016-05-17 04:33:49   
1454727  id3458201          2 2016-05-03 21:39:04 2016-05-03 22:42:51   
1454881  id1847851          2 2016-02-22 07:38:52 2016-02-22 07:38:56   

         passenger_count  pickup_longitude  pickup_latitude  \
991                    3        -